#Importando as bibliotecas.

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn.model_selection import RandomizedSearchCV
from math import sqrt
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

#Carregando os dados de treino e teste.

In [0]:
df_train = pd.read_csv('drive/My Drive/Colab Notebooks/train.csv')
df_test = pd.read_csv('drive/My Drive/Colab Notebooks/test.csv')

#Criando o DataFrame de submissão.

In [0]:
submission = pd.DataFrame()
submission[['User_ID','Product_ID']] = df_test[['User_ID','Product_ID']]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Transformação dos dados

Durante a análise exploratória de dados (EDA), ficou claro que na coluna 'Product_Category_1' temos dois outliers pontuais, vamos removê-los. No link a seguir você encontra mais detalhes 
https://github.com/karinnecristina/Data-Science/blob/master/Black_Friday/Estatistica_Descritiva.ipynb

In [5]:
df_train['Product_Category_1'].value_counts()

5     150933
1     140378
8     113925
11     24287
2      23864
6      20466
3      20213
4      11753
16      9828
15      6290
13      5549
10      5125
12      3947
7       3721
18      3125
20      2550
19      1603
14      1523
17       578
9        410
Name: Product_Category_1, dtype: int64

In [6]:
df_test['Product_Category_1'].value_counts()

5     65017
1     60321
8     48369
2     10192
11    10153
6      8860
3      8578
4      5003
16     4105
15     2694
13     2381
10     2248
12     1663
7      1624
18     1311
14      663
17      223
9       194
Name: Product_Category_1, dtype: int64

Como podemos observar acima só temos esses indíces no dataset de treino, então teremos que removê-los antes de concatenar esses datasets.

In [0]:
# Removendo os outliers do dataset de treino
df_train.drop(df_train[(df_train.Product_Category_1 == 19) | (df_train.Product_Category_1 == 20)].index,inplace=True)

Todas as transformações devem ser feitas em ambos datasets e afim de facilitar esse processo vamos concatena-los sem a variável target(Purchase), assim não precisamos fazer essa tarefa 2 vezes. 


In [8]:
train = df_train
test = df_test

df = pd.concat([train.drop('Purchase', axis=1),test],ignore_index = True, sort = False)
target = train['Purchase']

# Verificando a dimensão dos datasets.
train.shape, test.shape, df.shape

((545915, 12), (233599, 11), (779514, 11))

In [9]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN


Informações sobre as colunas

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779514 entries, 0 to 779513
Data columns (total 11 columns):
User_ID                       779514 non-null int64
Product_ID                    779514 non-null object
Gender                        779514 non-null object
Age                           779514 non-null object
Occupation                    779514 non-null int64
City_Category                 779514 non-null object
Stay_In_Current_City_Years    779514 non-null object
Marital_Status                779514 non-null int64
Product_Category_1            779514 non-null int64
Product_Category_2            537685 non-null float64
Product_Category_3            237858 non-null float64
dtypes: float64(2), int64(4), object(5)
memory usage: 65.4+ MB


Verificando a quantidade de valores nulos no dataset.

In [11]:
total = df.isnull().sum().sort_values(ascending=False)
percent = df.isnull().sum()/df.isnull().count().sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, sort=False, keys=['total', 'percent'])
missing_data[missing_data['percent']!=0]*100

,total,percent
Product_Category_3,54165600,69.486372
Product_Category_2,24182900,31.023048


Preeenchendo os valores nulos da coluna 'Product_Category_2' e transformando seus valores em inteiro.

In [0]:
df['Product_Category_2'] = df['Product_Category_2'].fillna(-999).astype(int)

Opa! Mais temos duas colunas com valores nulos e só estamos preenchendo uma, isso parece meio estranho. Vamos dar uma olhada nos valores únicos de cada coluna.

In [13]:
df.apply(lambda x: len(x.unique()))

User_ID                       5891
Product_ID                    3672
Gender                           2
Age                              7
Occupation                      21
City_Category                    3
Stay_In_Current_City_Years       5
Marital_Status                   2
Product_Category_1              18
Product_Category_2              18
Product_Category_3              16
dtype: int64

**Podemos observar que a coluna 'Product_Category_3' tem um número menor de valores únicos comparado com as outras duas colunas referentes a categoria do produto. Conforme foi informado na descrição do problema os produtos da categoria 2 e 3 podem pertencer a outra categoria também, mais isso não se aplica aos produtos da categoria  1 que é a categoria principal. Podemos observar que 2 produtos da categoria principal não pertence a categoria 3, então isso pode nos indicar que essa categoria de produtos não Influenciará no valor final da compra.**

Temos vários IDs(1000001,1000002,1000004 - que inclusive se repetem) e cada um 
desses IDs está relacionado com vários produtos.
Vamos verificar a quantidade de usuários por ID e a quantidade de produtos. Para efetuar essa contagem vamos utilizar o **transform** que faz a contagem mais retorna a mesma quantidade de dados diferentemente do aggregate que retorna uma versão reduzida dos dados. Por último vamos ver qual o valor médio de compra de cada produto.

In [0]:
# Número de usuários por ID.
df['User_count'] = df.User_ID.groupby(df.User_ID).transform('count')

# Quantidade de produtos.
df['Product_count'] = df.Product_ID.groupby(df.Product_ID).transform('count')

# Valor médio de compra de cada produto.
df['Product_mean'] = df['Product_ID'].map(target.groupby(train['Product_ID']).mean())
df['Product_mean'] = df['Product_mean'].replace(np.nan, 0)

Transformando os dados da coluna 'Age' em numéricos com a média aritmética entre os valores.

In [0]:
age_dict = {'0-17':17, '18-25':21, '26-35':30, '36-45':40, '46-50':48, '51-55':53, '55+':60}
df["Age"] = df["Age"].apply(lambda x: age_dict[x])

 Na coluna 'Stay_In_Current_City_Years' temos uma string acompanhada de um caracter especial, vamos trocar o valor dessa string por 5 já que ela está como '4+' ao meu ver isso significa que é mais de 4 por isso vou preencher com o valor 5 e transformar seus dados para inteiros.

In [0]:
df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].replace('4+', 5).astype(int)

Transformando as variáveis categóricas nominais em numéricas. Estou usando o **drop_first** para evitar o problema de Dummy Variable trap. Mais detalhes no site:https://medium.com/@saurav9786/dummy-variable-trap-c6d4a387f10a

In [0]:
df = pd.get_dummies(df, columns=['Gender','City_Category'], drop_first=True)

Visualizando se os dados foram preenchidos corretamente.

In [18]:
df.head()

,User_ID,Product_ID,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,User_count,Product_count,Product_mean,Gender_M,City_Category_B,City_Category_C
0,1000001,P00069042,17,10,2,0,3,-999,NaN,46,344,11870.863436,0,0,0
1,1000001,P00248942,17,10,2,0,1,6,14.0,46,845,16304.030981,0,0,0
2,1000001,P00087842,17,10,2,0,12,-999,NaN,46,150,1237.892157,0,0,0
3,1000001,P00085442,17,10,2,0,12,14,NaN,46,496,1455.140762,0,0,0
4,1000002,P00285442,60,16,5,0,8,-999,NaN,106,283,7692.763547,1,0,1


Agora vamos iremos remover as colunas 'User_ID' e 'Product_ID', pois elas só contém a identificação dos usuários e dos produtos, e a coluna 'Product_Category_3' que como vimos acima não tem uma relação total com a categoria principal e não serão necessárias no nosso modelo.



In [19]:
def remove_features(lista_features):
    for i in lista_features:
        df.drop(i, axis=1, inplace=True)
remove_features(['User_ID','Product_ID','Product_Category_3'])
df.head(n=20)

,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,User_count,Product_count,Product_mean,Gender_M,City_Category_B,City_Category_C
0,17,10,2,0,3,-999,46,344,11870.863436,0,0,0
1,17,10,2,0,1,6,46,845,16304.030981,0,0,0
2,17,10,2,0,12,-999,46,150,1237.892157,0,0,0
3,17,10,2,0,12,14,46,496,1455.140762,0,0,0
4,60,16,5,0,8,-999,106,283,7692.763547,1,0,1
5,30,15,3,0,1,2,41,863,14318.772358,1,0,0
6,48,7,2,1,1,8,14,2066,16899.227083,1,1,0
7,48,7,2,1,1,15,14,826,14169.618887,1,1,0
8,48,7,2,1,1,16,14,1304,15947.346491,1,1,0
9,30,20,1,1,8,-999,151,1139,8068.920555,1,0,0


Verificando se todas as variáveis foram transformadas em numéricas.

In [20]:
df.dtypes

Age                             int64
Occupation                      int64
Stay_In_Current_City_Years      int64
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2              int64
User_count                      int64
Product_count                   int64
Product_mean                  float64
Gender_M                        uint8
City_Category_B                 uint8
City_Category_C                 uint8
dtype: object

Depois de fazer todas as transformações necessárias nos dados
 vamos dividir o dataset em dados de treino e teste novamente.

In [0]:
X_train = df[:545915]
X_test = df[545915:]

Dataset de treino

In [22]:
X_train.head()

,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,User_count,Product_count,Product_mean,Gender_M,City_Category_B,City_Category_C
0,17,10,2,0,3,-999,46,344,11870.863436,0,0,0
1,17,10,2,0,1,6,46,845,16304.030981,0,0,0
2,17,10,2,0,12,-999,46,150,1237.892157,0,0,0
3,17,10,2,0,12,14,46,496,1455.140762,0,0,0
4,60,16,5,0,8,-999,106,283,7692.763547,1,0,1


Dataset de teste

In [23]:
X_test.head()

,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,User_count,Product_count,Product_mean,Gender_M,City_Category_B,City_Category_C
545915,48,7,2,1,1,11,14,1333,15781.118590,1,1,0
545916,30,17,0,0,3,5,84,371,11746.665354,1,0,1
545917,40,1,5,1,5,14,323,217,5731.338028,0,1,0
545918,40,1,5,1,4,9,323,23,1943.266667,0,1,0
545919,30,1,1,0,4,5,109,781,2585.590829,0,0,1


In [24]:
X_train.shape, X_test.shape

((545915, 12), (233599, 12))

In [25]:
target.values

array([ 8370, 15200,  1422, ...,  7852,  7159, 11640])

#Avaliação dos Algoritmos

Vamos criar pipelines afim de automatizar o processo de treinamento dos nossos modelos e aplicar a **Standartization** (padronização) ao conjunto de dados (colocando-os na mesma escala)
Nesta técnica, os dados serão transformados de modo que estejam com uma distrbuição normal, com média igual a zero e
desvio padrão igual a 1.

In [26]:
pipelines = []
pipelines.append(('Scaled-LR', Pipeline([('Scaler', StandardScaler()),('LR', LinearRegression())])))
pipelines.append(('Scaled-LASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('Scaled-EN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('Scaled-CART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
resultados = []
nomes = []

# Percorrendo cada um dos modelos
for nome, modelo in pipelines:
    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state = 7)
    cross_val_result = model_selection.cross_val_score(modelo, X_train, target, cv = kfold, scoring = 'neg_mean_squared_error')
    resultados.append(cross_val_result)
    nomes.append(nome)
    texto = "%s: %f (%f)" % (nome, np.sqrt(np.abs(cross_val_result)).mean(), np.sqrt(-cross_val_result).std())
    print(texto)

Scaled-LR: 2654.130781 (5.773837)
Scaled-LASSO: 2654.131221 (5.779685)
Scaled-EN: 3002.637471 (7.872833)
Scaled-CART: 3676.055662 (4.574383)


Vamos ver se conseguimos melhores resultados com os métodos Ensemble.

In [27]:
ensembles = []
ensembles.append(('Scaled-XGBoost', Pipeline([('Scaler', StandardScaler()),('XG', XGBRegressor(objective ='reg:squarederror'))])))
ensembles.append(('Scaled-AB', Pipeline([('Scaler', StandardScaler()),('AB', AdaBoostRegressor())])))
ensembles.append(('Scaled-GBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))
ensembles.append(('Scaled-RF', Pipeline([('Scaler', StandardScaler()),('RF', RandomForestRegressor())])))
ensembles.append(('Scaled-ET', Pipeline([('Scaler', StandardScaler()),('ET', ExtraTreesRegressor())])))
resultados = []
nomes = []

# Percorrendo cada um dos modelos
for nome, modelo in ensembles:
    kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state = 7)
    cross_val_result = model_selection.cross_val_score(modelo, X_train, target, cv = kfold, scoring = 'neg_mean_squared_error')
    resultados.append(cross_val_result)
    nomes.append(nome)
    texto = "%s: %f (%f)" % (nome, np.sqrt(np.abs(cross_val_result)).mean(), np.sqrt(-cross_val_result).std())
    print(texto)

Scaled-XGBoost: 2638.091322 (5.442491)
Scaled-AB: 2785.758834 (12.127388)
Scaled-GBM: 2638.022172 (5.470119)
Scaled-RF: 2738.599549 (7.288163)
Scaled-ET: 2847.718168 (8.866920)


O GradientBoostingRegressor e o XGBRegressor apresentaram a menor taxa de erro entre os modelos. De acordo com esse site https://www.vooo.pro/insights/um-tutorial-completo-sobre-a-modelagem-baseada-em-tree-arvore-do-zero-em-r-python/, o XGBRegressor possui algumas vantagens, nele você também encontra uma explicação e exemplos sobre os parâmetros que iremos utilizar, dito isso vamos otimizar esse modelo com o ajuste dos parâmetros.

Vamos agora fazer o tuning do modelo utilizando o RandomizedSearchCV, que de acordo com a documentação do scikit-learn tem o tempo de execução  
drasticamente menor que o GridSearchCV. Foi testado vários valores para os parâmetros e deixei aqui só os valores que apareceram com mais frequência (pra ajudar a diminuir o tempo de processamento).

In [28]:
# Definindo a escala
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

# Testando valores para o estimador.
valores_grid  = {'n_estimators' : [100,200,300,400], 'max_depth' : [7,9],'min_child_weight': [5,7], 'gamma': [0.1,0.3]}

# Criando o modelo
modelo = XGBRegressor(objective ='reg:squarederror', n_jobs=4)

# Definindo k
kfold = model_selection.KFold(5, True, random_state = 7)

# Testando a combinação de parâmetros
grid = RandomizedSearchCV(modelo, valores_grid, cv = kfold, scoring = 'neg_mean_squared_error')
grid_result = grid.fit(rescaledX, target)

# Print do resultado
print("Grid scores on development set:")
means = grid.cv_results_['mean_test_score'].round(5)
stds = grid.cv_results_['std_test_score'].round(5)

for mean, std, params in zip((means), stds, grid.cv_results_['params']):
    print(f'mean:{mean},std:{std},params:{params}')
print()
print(f'Melhor parâmetro:{grid.best_params_}, Score:{grid.best_score_}')

Grid scores on development set:
mean:-6397452.96592,std:21979.36254,params:{'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 7, 'gamma': 0.3}
mean:-6227883.27624,std:21787.09247,params:{'n_estimators': 200, 'min_child_weight': 7, 'max_depth': 9, 'gamma': 0.3}
mean:-6410785.51822,std:26743.96619,params:{'n_estimators': 100, 'min_child_weight': 7, 'max_depth': 9, 'gamma': 0.1}
mean:-6140429.36189,std:21939.04151,params:{'n_estimators': 400, 'min_child_weight': 5, 'max_depth': 9, 'gamma': 0.1}
mean:-6303622.19197,std:16272.26487,params:{'n_estimators': 300, 'min_child_weight': 5, 'max_depth': 7, 'gamma': 0.3}
mean:-6410785.51822,std:26743.96619,params:{'n_estimators': 100, 'min_child_weight': 7, 'max_depth': 9, 'gamma': 0.3}
mean:-6228978.25806,std:18539.17347,params:{'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 9, 'gamma': 0.3}
mean:-6604243.9681,std:28666.6868,params:{'n_estimators': 100, 'min_child_weight': 5, 'max_depth': 7, 'gamma': 0.1}
mean:-6417770.80977,std:32

**É claro que não me deixei levar pelas vantagens do XGBRegressor e também fiz o tuning do  GradientBoostingRegressor, e de fato obtive uma melhor pontuação com o XGBRegressor. 
Como o processo é um pouco demorado optei por mostrar somente um dos processos.**

#Preparando a versão final do modelo

In [30]:
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
modelo_xbg =  XGBRegressor(objective ='reg:squarederror', n_estimators = 400,
                                                          min_child_weight=5,
                                                          max_depth=9, 
                                                          gamma=0.1, 
                                                          random_state=7,
                                                          n_jobs=4)
                                                          
modelo_xbg.fit(rescaledX, target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=9, min_child_weight=5, missing=None, n_estimators=400,
             n_jobs=4, nthread=None, objective='reg:squarederror',
             random_state=7, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

#Aplicando o modelo aos dados de teste

In [31]:
rescaledValidationX = scaler.transform(X_test)
previsoes = modelo_xbg.predict(rescaledValidationX)
previsoes

array([14411.546 , 11803.302 ,  6551.573 , ...,  9042.282 , 16667.99  ,
        2944.9668], dtype=float32)

#Visualizando os resultados

In [32]:
submission['User_ID','Product_ID'] = np.around(previsoes,2)
submission = submission.rename(columns={('User_ID','Product_ID'):'Purchase'})
submission.head()

,User_ID,Product_ID,Purchase
0,1000004,P00128942,14411.549805
1,1000009,P00113442,11803.299805
2,1000010,P00288442,6551.569824
3,1000010,P00145342,2945.649902
4,1000011,P00053842,2615.340088


#Submissão

In [0]:
submission.to_csv("sample_submission_LMg97w5.csv", index=False)

**Estou no 257° lugar com o RMSE de 2485.1888593737,  "Em cada experiência se aprende uma lição"**
